<a href="https://colab.research.google.com/github/aromanenko/ATSF/blob/wip/hw3_solution_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
%matplotlib inline

In [5]:
#Загружаем данные, содержащие информацию по всем товарам во всех магазинах за все периоды времени (тренировочные + для которых необходимо предсказать спрос)
all_data = pd.read_csv('train.csv', delimiter=',')
all_data

,Unnamed: 0,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG
0,0,40369,309,2016-12-19,29.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,40370,309,2016-12-19,64.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,40372,309,2016-12-19,32.0,NaN,NaN,NaN,NaN,NaN,NaN
3,3,40373,309,2016-12-19,10.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,46272,309,2016-12-19,15.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
35339,35537,40370,1380,2019-12-30,NaN,0.0,0.0,1000.00,1000.0,0.0,1.0
35340,35538,40372,1380,2019-12-30,NaN,0.0,0.0,2000.00,2000.0,0.0,1.0
35341,35539,40373,1380,2019-12-30,NaN,0.0,0.0,3000.00,3000.0,0.0,1.0
35342,35540,46272,1380,2019-12-30,NaN,1.0,0.0,284.29,199.0,0.0,1.0


In [6]:
#Сразу переименуем столбец "Unnamed: 0" в id
all_data.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
all_data

,id,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG
0,0,40369,309,2016-12-19,29.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,40370,309,2016-12-19,64.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,40372,309,2016-12-19,32.0,NaN,NaN,NaN,NaN,NaN,NaN
3,3,40373,309,2016-12-19,10.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,46272,309,2016-12-19,15.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
35339,35537,40370,1380,2019-12-30,NaN,0.0,0.0,1000.00,1000.0,0.0,1.0
35340,35538,40372,1380,2019-12-30,NaN,0.0,0.0,2000.00,2000.0,0.0,1.0
35341,35539,40373,1380,2019-12-30,NaN,0.0,0.0,3000.00,3000.0,0.0,1.0
35342,35540,46272,1380,2019-12-30,NaN,1.0,0.0,284.29,199.0,0.0,1.0


In [7]:
#Посмотрим на уникальные значения категориальных объясняющих переменных и удалим те,
#у которых уникальное значение (не пустое) только одно, т.к. они не дают никакой пользы при обучении модели
print(all_data['PROMO1_FLAG'].unique())
print(all_data['PROMO2_FLAG'].unique()) #не используем данную переменную, т.к. значения только 0 и NaN
print(all_data['NUM_CONSULTANT'].unique()) #не используем данную переменную, т.к. значения только 0 и NaN
print(all_data['AUTORIZATION_FLAG'].unique())

[nan  1.  0.  2.]
[nan  0.]
[nan  0.]
[nan  1.  0.]


In [8]:
del all_data['PROMO2_FLAG']
del all_data['NUM_CONSULTANT']
all_data

,id,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG
0,0,40369,309,2016-12-19,29.0,NaN,NaN,NaN,NaN
1,1,40370,309,2016-12-19,64.0,NaN,NaN,NaN,NaN
2,2,40372,309,2016-12-19,32.0,NaN,NaN,NaN,NaN
3,3,40373,309,2016-12-19,10.0,NaN,NaN,NaN,NaN
4,4,46272,309,2016-12-19,15.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
35339,35537,40370,1380,2019-12-30,NaN,0.0,1000.00,1000.0,1.0
35340,35538,40372,1380,2019-12-30,NaN,0.0,2000.00,2000.0,1.0
35341,35539,40373,1380,2019-12-30,NaN,0.0,3000.00,3000.0,1.0
35342,35540,46272,1380,2019-12-30,NaN,1.0,284.29,199.0,1.0


In [9]:
#Поработаем с пропущенными значениями
#Пропуски в категориальных переменных заполним соответствующими модами:
all_data['PROMO1_FLAG'] = all_data['PROMO1_FLAG'].fillna(all_data['PROMO1_FLAG'].mode()[0])
all_data['AUTORIZATION_FLAG'] = all_data['AUTORIZATION_FLAG'].fillna(all_data['AUTORIZATION_FLAG'].mode()[0])
all_data

,id,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG
0,0,40369,309,2016-12-19,29.0,0.0,NaN,NaN,1.0
1,1,40370,309,2016-12-19,64.0,0.0,NaN,NaN,1.0
2,2,40372,309,2016-12-19,32.0,0.0,NaN,NaN,1.0
3,3,40373,309,2016-12-19,10.0,0.0,NaN,NaN,1.0
4,4,46272,309,2016-12-19,15.0,0.0,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...
35339,35537,40370,1380,2019-12-30,NaN,0.0,1000.00,1000.0,1.0
35340,35538,40372,1380,2019-12-30,NaN,0.0,2000.00,2000.0,1.0
35341,35539,40373,1380,2019-12-30,NaN,0.0,3000.00,3000.0,1.0
35342,35540,46272,1380,2019-12-30,NaN,1.0,284.29,199.0,1.0


In [10]:
#Пропуски в числовых переменных будем заполнять по каждой паре "товар + магазин". Для каждой пары в каждый временной период с NaN в переменных
#PRICE_REGULAR и PRICE_AFTER_DISC будем всавлять медианную цену среди цен на этот товар на данную дату в других магазинах, в которых цена известна.
#При этом удалим предварительно из выборки 309 магазин, т.к. ни по одному товару ни в одну из дат в этом магазине
#неизвестны цены (переменные PRICE_REGULAR и PRICE_AFTER_DISC)
all_data[all_data['store_location_rk'] == 309].sort_values(by='product_rk')

,id,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG
0,0,40369,309,2016-12-19,29.0,0.0,NaN,NaN,1.0
5,5,40369,309,2016-12-26,127.0,0.0,NaN,NaN,1.0
10,10,40369,309,2017-01-02,50.0,0.0,NaN,NaN,1.0
6,6,40370,309,2016-12-26,181.0,0.0,NaN,NaN,1.0
11,11,40370,309,2017-01-02,70.0,0.0,NaN,NaN,1.0
1,1,40370,309,2016-12-19,64.0,0.0,NaN,NaN,1.0
7,7,40372,309,2016-12-26,88.0,0.0,NaN,NaN,1.0
2,2,40372,309,2016-12-19,32.0,0.0,NaN,NaN,1.0
12,12,40372,309,2017-01-02,30.0,0.0,NaN,NaN,1.0
3,3,40373,309,2016-12-19,10.0,0.0,NaN,NaN,1.0


In [11]:
all_data = all_data[all_data['store_location_rk'] != 309]
all_data #удалилось 15 наблюдений

,id,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG
15,15,40369,317,2016-12-19,50.0,0.0,NaN,NaN,1.0
16,16,40370,317,2016-12-19,44.0,0.0,NaN,NaN,1.0
17,17,40372,317,2016-12-19,13.0,0.0,NaN,NaN,1.0
18,18,40373,317,2016-12-19,6.0,0.0,NaN,NaN,1.0
19,19,46272,317,2016-12-19,34.0,0.0,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...
35339,35537,40370,1380,2019-12-30,NaN,0.0,1000.00,1000.0,1.0
35340,35538,40372,1380,2019-12-30,NaN,0.0,2000.00,2000.0,1.0
35341,35539,40373,1380,2019-12-30,NaN,0.0,3000.00,3000.0,1.0
35342,35540,46272,1380,2019-12-30,NaN,1.0,284.29,199.0,1.0


In [12]:
#Удалим также такие пары "товар+день", у которых вообще не известна цена ни в одном магазине. Выведем для начала такие пары (их пять)
n = 0
listt = []
for i in all_data['product_rk'].unique():
  for d in all_data[all_data['product_rk'] == i]['period_start_dt'].unique():
    if len(all_data[(all_data['product_rk'] == i) & (all_data['period_start_dt'] == d)]) == all_data[(all_data['product_rk'] == i) & (all_data['period_start_dt'] == d)]['PRICE_REGULAR'].isnull().sum():
      n += 1
      listt.append([i, d])

print(n)
print(listt)

5
[[np.int64(40369), '2016-12-19'], [np.int64(40370), '2016-12-19'], [np.int64(40372), '2016-12-19'], [np.int64(40373), '2016-12-19'], [np.int64(46272), '2016-12-19']]


In [13]:
all_data = all_data[(all_data['product_rk'] != 40369) | (all_data['period_start_dt'] != '2016-12-19')]
all_data = all_data[(all_data['product_rk'] != 40370) | (all_data['period_start_dt'] != '2016-12-19')]
all_data = all_data[(all_data['product_rk'] != 40372) | (all_data['period_start_dt'] != '2016-12-19')]
all_data = all_data[(all_data['product_rk'] != 40373) | (all_data['period_start_dt'] != '2016-12-19')]
all_data = all_data[(all_data['product_rk'] != 46272) | (all_data['period_start_dt'] != '2016-12-19')]

In [14]:
#Проверка
n = 0
listt = []
for i in all_data['product_rk'].unique():
  for d in all_data[all_data['product_rk'] == i]['period_start_dt'].unique():
    if len(all_data[(all_data['product_rk'] == i) & (all_data['period_start_dt'] == d)]) == all_data[(all_data['product_rk'] == i) & (all_data['period_start_dt'] == d)]['PRICE_REGULAR'].isnull().sum():
      n += 1
      listt.append([i, d])

print(n)
print(listt)
#Таким образом, на 2016-12-19 остался только один товар №96212

0
[]


In [15]:
#Проверка:
a = 0
for i in all_data['product_rk'].unique():
  for d in all_data[all_data['product_rk'] == i]['period_start_dt'].unique():
    a += 1

a #919 пар вместо 924

919

In [16]:
#Выведем для проверки, что были NaN в переменных PRICE_REGULAR и PRICE_AFTER_DISC, а после следующих двух ячеек проверим, что они исчезли
all_data[(all_data['product_rk'] == 96212) & (all_data['period_start_dt'] == '2019-11-04')]

,id,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG
889,889,96212,317,2019-11-04,3.966667,1.0,70.0,49.000000,1.0
1812,1818,96212,355,2019-11-04,7.000000,1.0,70.0,49.000000,1.0
2735,2747,96212,380,2019-11-04,3.000000,1.0,70.0,49.000000,1.0
3659,3677,96212,425,2019-11-04,5.366667,1.0,70.0,49.000000,1.0
4582,4606,96212,453,2019-11-04,3.733333,1.0,70.0,49.000000,1.0
5501,5531,96212,504,2019-11-04,9.000000,1.0,70.0,49.000000,1.0
6425,6460,96212,517,2019-11-04,0.000000,1.0,70.0,49.000000,1.0
7348,7388,96212,525,2019-11-04,4.200000,1.0,70.0,49.000000,1.0
8272,8318,96212,533,2019-11-04,0.000000,0.0,NaN,NaN,0.0
9194,9240,96212,535,2019-11-04,4.000000,1.0,70.0,49.000000,1.0


In [17]:
#Заполняем пропуски в цене товара i (PRICE_REGULAR) на дату d медианной ценой этого товара по всем магазинам, в которых он продается, на данную дату d
for i in all_data['product_rk'].unique():
  # print(i)
  for d in all_data[all_data['product_rk'] == i]['period_start_dt'].unique():
    # print(d)
    # print(all_data[(all_data['product_rk'] == i) & (all_data['period_start_dt'] == d)]['PRICE_REGULAR'].median())
    all_data.loc[(all_data['product_rk'] == i) & (all_data['period_start_dt'] == d), ['PRICE_REGULAR']] = all_data.loc[(all_data['product_rk'] == i) & (all_data['period_start_dt'] == d)]['PRICE_REGULAR'].fillna(all_data.loc[(all_data['product_rk'] == i) & (all_data['period_start_dt'] == d)]['PRICE_REGULAR'].median())
    # print(all_data[(all_data['product_rk'] == i) & (all_data['period_start_dt'] == d)]['PRICE_REGULAR'])

In [18]:
#Заполняем аналогично пропуски в переменной PRICE_AFTER_DISC
for i in all_data['product_rk'].unique():
  # print(i)
  for d in all_data[all_data['product_rk'] == i]['period_start_dt'].unique():
    # print(d)
    # print(all_data[(all_data['product_rk'] == i) & (all_data['period_start_dt'] == d)]['PRICE_AFTER_DISC'].median())
    all_data.loc[(all_data['product_rk'] == i) & (all_data['period_start_dt'] == d), ['PRICE_AFTER_DISC']] = all_data.loc[(all_data['product_rk'] == i) & (all_data['period_start_dt'] == d)]['PRICE_AFTER_DISC'].fillna(all_data.loc[(all_data['product_rk'] == i) & (all_data['period_start_dt'] == d)]['PRICE_AFTER_DISC'].median())
    # print(all_data[(all_data['product_rk'] == i) & (all_data['period_start_dt'] == d)]['PRICE_AFTER_DISC'])

In [19]:
#Выведем для проверки, что все NaN в переменных PRICE_REGULAR и PRICE_AFTER_DISC заменились на соответствующие медианы
all_data[(all_data['product_rk'] == 96212) & (all_data['period_start_dt'] == '2019-11-04')]

,id,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG
889,889,96212,317,2019-11-04,3.966667,1.0,70.0,49.000000,1.0
1812,1818,96212,355,2019-11-04,7.000000,1.0,70.0,49.000000,1.0
2735,2747,96212,380,2019-11-04,3.000000,1.0,70.0,49.000000,1.0
3659,3677,96212,425,2019-11-04,5.366667,1.0,70.0,49.000000,1.0
4582,4606,96212,453,2019-11-04,3.733333,1.0,70.0,49.000000,1.0
5501,5531,96212,504,2019-11-04,9.000000,1.0,70.0,49.000000,1.0
6425,6460,96212,517,2019-11-04,0.000000,1.0,70.0,49.000000,1.0
7348,7388,96212,525,2019-11-04,4.200000,1.0,70.0,49.000000,1.0
8272,8318,96212,533,2019-11-04,0.000000,0.0,70.0,49.000000,0.0
9194,9240,96212,535,2019-11-04,4.000000,1.0,70.0,49.000000,1.0


In [20]:
#Проверка:
all_data['PRICE_REGULAR'].isnull().any()

np.False_

In [21]:
# Преобразуем столбец 'period_start_dt' в формат datetime
all_data['period_start_dt'] = pd.to_datetime(all_data['period_start_dt'], errors='coerce')

In [22]:
all_data

,id,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG
20,20,40369,317,2016-12-26,65.0,1.0,500.00,500.0,1.0
21,21,40370,317,2016-12-26,83.0,1.0,1000.00,1000.0,1.0
22,22,40372,317,2016-12-26,30.0,1.0,2000.00,2000.0,1.0
23,23,40373,317,2016-12-26,7.0,1.0,3000.00,3000.0,1.0
24,24,46272,317,2016-12-26,35.0,1.0,157.00,157.0,1.0
...,...,...,...,...,...,...,...,...,...
35339,35537,40370,1380,2019-12-30,NaN,0.0,1000.00,1000.0,1.0
35340,35538,40372,1380,2019-12-30,NaN,0.0,2000.00,2000.0,1.0
35341,35539,40373,1380,2019-12-30,NaN,0.0,3000.00,3000.0,1.0
35342,35540,46272,1380,2019-12-30,NaN,1.0,284.29,199.0,1.0


In [23]:
missing_per_column = all_data.isnull().sum()
missing_per_column

id                      0
product_rk              0
store_location_rk       0
period_start_dt         0
demand               1200
PROMO1_FLAG             0
PRICE_REGULAR           0
PRICE_AFTER_DISC        0
AUTORIZATION_FLAG       0
dtype: int64

In [24]:
#перекодируем даты. Создадим три переменных ind_of_year, ind_of_month и ind_of_day, которые будут отражать год, месяц и день наблюдения соответственно

# all_data['period_start_dt'] = pd.to_datetime(all_data['period_start_dt'], dayfirst =False)
all_data["ind_of_year"] = all_data.period_start_dt.dt.year
all_data["ind_of_month"] = all_data.period_start_dt.dt.month
all_data["ind_of_day"] = all_data.period_start_dt.dt.day
all_data.head()

,id,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG,ind_of_year,ind_of_month,ind_of_day
20,20,40369,317,2016-12-26,65.0,1.0,500.0,500.0,1.0,2016,12,26
21,21,40370,317,2016-12-26,83.0,1.0,1000.0,1000.0,1.0,2016,12,26
22,22,40372,317,2016-12-26,30.0,1.0,2000.0,2000.0,1.0,2016,12,26
23,23,40373,317,2016-12-26,7.0,1.0,3000.0,3000.0,1.0,2016,12,26
24,24,46272,317,2016-12-26,35.0,1.0,157.0,157.0,1.0,2016,12,26


In [25]:
#пример
all_data.iloc[300:600]

,id,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG,ind_of_year,ind_of_month,ind_of_day
320,320,40369,317,2018-01-15,12.000000,0.0,500.0,500.0,1.0,2018,1,15
321,321,40370,317,2018-01-15,8.000000,0.0,1000.0,1000.0,1.0,2018,1,15
322,322,40372,317,2018-01-15,4.000000,0.0,2000.0,2000.0,1.0,2018,1,15
323,323,40373,317,2018-01-15,2.000000,0.0,3000.0,3000.0,1.0,2018,1,15
324,324,46272,317,2018-01-15,3.000000,0.0,239.0,239.0,1.0,2018,1,15
...,...,...,...,...,...,...,...,...,...,...,...,...
615,615,40370,317,2018-12-24,69.000000,0.0,1000.0,1000.0,1.0,2018,12,24
616,616,40372,317,2018-12-24,12.000000,0.0,2000.0,2000.0,1.0,2018,12,24
617,617,40373,317,2018-12-24,16.000000,0.0,3000.0,3000.0,1.0,2018,12,24
618,618,46272,317,2018-12-24,18.266667,1.0,329.0,98.7,1.0,2018,12,24


In [34]:
all_data = all_data.rename(columns={'period_start_dt': 'Date'})
all_data = all_data.rename(columns={'store_location_rk': 'Store_id'})
all_data = all_data.rename(columns={'product_rk': 'SKU_id'})
all_data = all_data.rename(columns={'PROMO1_FLAG': 'Promo'})
all_data = all_data.rename(columns={'demand': 'Demand'})
all_data = all_data.rename(columns={'PRICE_REGULAR': 'Regular_Price'})
all_data = all_data.rename(columns={'PRICE_AFTER_DISC': 'Promo_Price'})

In [35]:
all_data

,id,SKU_id,Store_id,Date,Demand,Promo,Regular_Price,Promo_Price,AUTORIZATION_FLAG,ind_of_year,ind_of_month,ind_of_day
20,20,40369,317,2016-12-26,65.0,1.0,500.00,500.0,1.0,2016,12,26
21,21,40370,317,2016-12-26,83.0,1.0,1000.00,1000.0,1.0,2016,12,26
22,22,40372,317,2016-12-26,30.0,1.0,2000.00,2000.0,1.0,2016,12,26
23,23,40373,317,2016-12-26,7.0,1.0,3000.00,3000.0,1.0,2016,12,26
24,24,46272,317,2016-12-26,35.0,1.0,157.00,157.0,1.0,2016,12,26
...,...,...,...,...,...,...,...,...,...,...,...,...
35339,35537,40370,1380,2019-12-30,NaN,0.0,1000.00,1000.0,1.0,2019,12,30
35340,35538,40372,1380,2019-12-30,NaN,0.0,2000.00,2000.0,1.0,2019,12,30
35341,35539,40373,1380,2019-12-30,NaN,0.0,3000.00,3000.0,1.0,2019,12,30
35342,35540,46272,1380,2019-12-30,NaN,1.0,284.29,199.0,1.0,2019,12,30


In [37]:
def process(all_data):
    all_data['Date'] = pd.to_datetime(all_data['Date'], dayfirst=True)
    all_data.set_index(['Date'], inplace=True)
    all_data['Promo'] = all_data['Promo'].fillna(0)
    all_data.reset_index().set_index(['SKU_id', 'Store_id', 'Date'], inplace=True)
    all_data['Regular_Price'] = all_data['Regular_Price'].ffill().bfill()
    all_data.reset_index().set_index(['Date'], inplace=True)

    # add actual price (promo price when promo occurred or regular price otherwise)
    all_data['Actual_Price'] = all_data['Promo_Price'].combine_first(all_data['Regular_Price'])
    all_data['Promo_percent'] = (1 - (all_data['Actual_Price'] / all_data['Regular_Price']))
    all_data = all_data.drop('Promo_Price', axis=1)

    all_data.reset_index(inplace=True)
    all_data["weekday"] = all_data['Date'].dt.weekday
    all_data["monthday"] = all_data['Date'].dt.day
    all_data['is_weekend'] = all_data['weekday'].isin([5, 6]) * 1
    all_data['month_period'] = 0
    all_data.loc[all_data['monthday'] >= 15, 'month_period'] = 1

    ## base feature

    # Использование transform для корректного расширенного среднего значения
    all_data['demand_expanding_mean'] = (
        all_data.groupby(['Store_id', 'SKU_id'])['Demand']
            .transform(lambda x: x.expanding().mean())
    )

    # Применяем shift на результат transform
    all_data['demand_expanding_mean'] = (
        all_data.groupby(['Store_id', 'SKU_id'])['demand_expanding_mean']
            .transform(lambda x: x.shift(14))
    )

    return all_data

In [39]:
from src.features.aggregations import *

ModuleNotFoundError: No module named 'src'

In [40]:
data = process(all_data)

In [66]:
data

,id,SKU_id,Store_id,Demand,Promo,Regular_Price,AUTORIZATION_FLAG,ind_of_year,ind_of_month,ind_of_day,Actual_Price,Promo_percent,weekday,monthday,is_weekend,month_period,demand_expanding_mean,weekday_name
0,20,40369,317,65.0,1.0,500.00,1.0,2016,12,26,500.0,0.000000,0,26,0,1,15.954331,Monday
1,21,40370,317,83.0,1.0,1000.00,1.0,2016,12,26,1000.0,0.000000,0,26,0,1,15.954331,Monday
2,22,40372,317,30.0,1.0,2000.00,1.0,2016,12,26,2000.0,0.000000,0,26,0,1,15.954331,Monday
3,23,40373,317,7.0,1.0,3000.00,1.0,2016,12,26,3000.0,0.000000,0,26,0,1,15.954331,Monday
4,24,46272,317,35.0,1.0,157.00,1.0,2016,12,26,157.0,0.000000,0,26,0,1,15.954331,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35154,35537,40370,1380,NaN,0.0,1000.00,1.0,2019,12,30,1000.0,0.000000,0,30,0,1,12.170732,Monday
35155,35538,40372,1380,NaN,0.0,2000.00,1.0,2019,12,30,2000.0,0.000000,0,30,0,1,6.560976,Monday
35156,35539,40373,1380,NaN,0.0,3000.00,1.0,2019,12,30,3000.0,0.000000,0,30,0,1,4.073171,Monday
35157,35540,46272,1380,NaN,1.0,284.29,1.0,2019,12,30,199.0,0.300011,0,30,0,1,4.194444,Monday


In [42]:
# data['weekday_name'] = data['Date'].dt.strftime('%A')

In [44]:
data['demand_expanding_mean'].fillna(data['demand_expanding_mean'].mean(), inplace=True)

/tmp/ipykernel_552303/4109259693.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['demand_expanding_mean'].fillna(data['demand_expanding_mean'].mean(), inplace=True)


In [45]:
data

,Date,id,SKU_id,Store_id,Demand,Promo,Regular_Price,AUTORIZATION_FLAG,ind_of_year,ind_of_month,ind_of_day,Actual_Price,Promo_percent,weekday,monthday,is_weekend,month_period,demand_expanding_mean,weekday_name
0,2016-12-26,20,40369,317,65.0,1.0,500.00,1.0,2016,12,26,500.0,0.000000,0,26,0,1,15.954331,Monday
1,2016-12-26,21,40370,317,83.0,1.0,1000.00,1.0,2016,12,26,1000.0,0.000000,0,26,0,1,15.954331,Monday
2,2016-12-26,22,40372,317,30.0,1.0,2000.00,1.0,2016,12,26,2000.0,0.000000,0,26,0,1,15.954331,Monday
3,2016-12-26,23,40373,317,7.0,1.0,3000.00,1.0,2016,12,26,3000.0,0.000000,0,26,0,1,15.954331,Monday
4,2016-12-26,24,46272,317,35.0,1.0,157.00,1.0,2016,12,26,157.0,0.000000,0,26,0,1,15.954331,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35154,2019-12-30,35537,40370,1380,NaN,0.0,1000.00,1.0,2019,12,30,1000.0,0.000000,0,30,0,1,12.170732,Monday
35155,2019-12-30,35538,40372,1380,NaN,0.0,2000.00,1.0,2019,12,30,2000.0,0.000000,0,30,0,1,6.560976,Monday
35156,2019-12-30,35539,40373,1380,NaN,0.0,3000.00,1.0,2019,12,30,3000.0,0.000000,0,30,0,1,4.073171,Monday
35157,2019-12-30,35540,46272,1380,NaN,1.0,284.29,1.0,2019,12,30,199.0,0.300011,0,30,0,1,4.194444,Monday


In [47]:
#и теперь удалим переменную period_start_dt за ненадобностью
del data['Date']
data

,id,SKU_id,Store_id,Demand,Promo,Regular_Price,AUTORIZATION_FLAG,ind_of_year,ind_of_month,ind_of_day,Actual_Price,Promo_percent,weekday,monthday,is_weekend,month_period,demand_expanding_mean,weekday_name
0,20,40369,317,65.0,1.0,500.00,1.0,2016,12,26,500.0,0.000000,0,26,0,1,15.954331,Monday
1,21,40370,317,83.0,1.0,1000.00,1.0,2016,12,26,1000.0,0.000000,0,26,0,1,15.954331,Monday
2,22,40372,317,30.0,1.0,2000.00,1.0,2016,12,26,2000.0,0.000000,0,26,0,1,15.954331,Monday
3,23,40373,317,7.0,1.0,3000.00,1.0,2016,12,26,3000.0,0.000000,0,26,0,1,15.954331,Monday
4,24,46272,317,35.0,1.0,157.00,1.0,2016,12,26,157.0,0.000000,0,26,0,1,15.954331,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35154,35537,40370,1380,NaN,0.0,1000.00,1.0,2019,12,30,1000.0,0.000000,0,30,0,1,12.170732,Monday
35155,35538,40372,1380,NaN,0.0,2000.00,1.0,2019,12,30,2000.0,0.000000,0,30,0,1,6.560976,Monday
35156,35539,40373,1380,NaN,0.0,3000.00,1.0,2019,12,30,3000.0,0.000000,0,30,0,1,4.073171,Monday
35157,35540,46272,1380,NaN,1.0,284.29,1.0,2019,12,30,199.0,0.300011,0,30,0,1,4.194444,Monday


In [67]:
del data['weekday_name']
data

,id,SKU_id,Store_id,Demand,Promo,Regular_Price,AUTORIZATION_FLAG,ind_of_year,ind_of_month,ind_of_day,Actual_Price,Promo_percent,weekday,monthday,is_weekend,month_period,demand_expanding_mean
0,20,40369,317,65.0,1.0,500.00,1.0,2016,12,26,500.0,0.000000,0,26,0,1,15.954331
1,21,40370,317,83.0,1.0,1000.00,1.0,2016,12,26,1000.0,0.000000,0,26,0,1,15.954331
2,22,40372,317,30.0,1.0,2000.00,1.0,2016,12,26,2000.0,0.000000,0,26,0,1,15.954331
3,23,40373,317,7.0,1.0,3000.00,1.0,2016,12,26,3000.0,0.000000,0,26,0,1,15.954331
4,24,46272,317,35.0,1.0,157.00,1.0,2016,12,26,157.0,0.000000,0,26,0,1,15.954331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35154,35537,40370,1380,NaN,0.0,1000.00,1.0,2019,12,30,1000.0,0.000000,0,30,0,1,12.170732
35155,35538,40372,1380,NaN,0.0,2000.00,1.0,2019,12,30,2000.0,0.000000,0,30,0,1,6.560976
35156,35539,40373,1380,NaN,0.0,3000.00,1.0,2019,12,30,3000.0,0.000000,0,30,0,1,4.073171
35157,35540,46272,1380,NaN,1.0,284.29,1.0,2019,12,30,199.0,0.300011,0,30,0,1,4.194444


In [68]:
#Подготовим данные для обучения модели
#Поделим данные на трейн и тест
data_train = data[data['Demand'].isnull() == False]
data_train #получаем 33959 наблюдений

,id,SKU_id,Store_id,Demand,Promo,Regular_Price,AUTORIZATION_FLAG,ind_of_year,ind_of_month,ind_of_day,Actual_Price,Promo_percent,weekday,monthday,is_weekend,month_period,demand_expanding_mean
0,20,40369,317,65.0,1.0,500.000000,1.0,2016,12,26,500.000000,0.000000,0,26,0,1,15.954331
1,21,40370,317,83.0,1.0,1000.000000,1.0,2016,12,26,1000.000000,0.000000,0,26,0,1,15.954331
2,22,40372,317,30.0,1.0,2000.000000,1.0,2016,12,26,2000.000000,0.000000,0,26,0,1,15.954331
3,23,40373,317,7.0,1.0,3000.000000,1.0,2016,12,26,3000.000000,0.000000,0,26,0,1,15.954331
4,24,46272,317,35.0,1.0,157.000000,1.0,2016,12,26,157.000000,0.000000,0,26,0,1,15.954331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35124,35507,40370,1380,24.0,0.0,1000.000000,1.0,2019,11,25,1000.000000,0.000000,0,25,0,1,12.638889
35125,35508,40372,1380,11.0,0.0,2000.000000,1.0,2019,11,25,2000.000000,0.000000,0,25,0,1,6.722222
35126,35509,40373,1380,3.0,0.0,3000.000000,1.0,2019,11,25,3000.000000,0.000000,0,25,0,1,4.166667
35127,35510,46272,1380,0.0,1.0,284.290000,1.0,2019,11,25,199.000000,0.300011,0,25,0,1,4.451613


In [69]:
data_test = data[data['Demand'].isnull()]
#Сразу переименуем столбец "demand" в тестовой выборке на "predicted"
data_test.rename(columns={'Demand': 'predicted'}, inplace=True)
data_test #получаем 1200 наблюдений

/tmp/ipykernel_552303/744666343.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test.rename(columns={'Demand': 'predicted'}, inplace=True)


,id,SKU_id,Store_id,predicted,Promo,Regular_Price,AUTORIZATION_FLAG,ind_of_year,ind_of_month,ind_of_day,Actual_Price,Promo_percent,weekday,monthday,is_weekend,month_period,demand_expanding_mean
888,908,40369,317,NaN,0.0,500.00,1.0,2019,12,2,500.0,0.000000,0,2,0,0,17.912489
889,909,40370,317,NaN,0.0,1000.00,1.0,2019,12,2,1000.0,0.000000,0,2,0,0,20.153431
890,910,40372,317,NaN,0.0,2000.00,1.0,2019,12,2,2000.0,0.000000,0,2,0,0,6.938798
891,911,40373,317,NaN,0.0,3000.00,1.0,2019,12,2,3000.0,0.000000,0,2,0,0,4.283099
892,912,46272,317,NaN,1.0,284.29,1.0,2019,12,2,199.0,0.300011,0,2,0,0,5.092526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35154,35537,40370,1380,NaN,0.0,1000.00,1.0,2019,12,30,1000.0,0.000000,0,30,0,1,12.170732
35155,35538,40372,1380,NaN,0.0,2000.00,1.0,2019,12,30,2000.0,0.000000,0,30,0,1,6.560976
35156,35539,40373,1380,NaN,0.0,3000.00,1.0,2019,12,30,3000.0,0.000000,0,30,0,1,4.073171
35157,35540,46272,1380,NaN,1.0,284.29,1.0,2019,12,30,199.0,0.300011,0,30,0,1,4.194444


In [70]:
X = data_train.drop(['id', 'Demand'], axis=1)
y = data_train['Demand']

In [71]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33959 entries, 0 to 35128
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   SKU_id                 33959 non-null  int64  
 1   Store_id               33959 non-null  int64  
 2   Promo                  33959 non-null  float64
 3   Regular_Price          33959 non-null  float64
 4   AUTORIZATION_FLAG      33959 non-null  float64
 5   ind_of_year            33959 non-null  int32  
 6   ind_of_month           33959 non-null  int32  
 7   ind_of_day             33959 non-null  int32  
 8   Actual_Price           33959 non-null  float64
 9   Promo_percent          33959 non-null  float64
 10  weekday                33959 non-null  int32  
 11  monthday               33959 non-null  int32  
 12  is_weekend             33959 non-null  int64  
 13  month_period           33959 non-null  int64  
 14  demand_expanding_mean  33959 non-null  float64
dtypes: floa

In [72]:
X

,SKU_id,Store_id,Promo,Regular_Price,AUTORIZATION_FLAG,ind_of_year,ind_of_month,ind_of_day,Actual_Price,Promo_percent,weekday,monthday,is_weekend,month_period,demand_expanding_mean
0,40369,317,1.0,500.000000,1.0,2016,12,26,500.000000,0.000000,0,26,0,1,15.954331
1,40370,317,1.0,1000.000000,1.0,2016,12,26,1000.000000,0.000000,0,26,0,1,15.954331
2,40372,317,1.0,2000.000000,1.0,2016,12,26,2000.000000,0.000000,0,26,0,1,15.954331
3,40373,317,1.0,3000.000000,1.0,2016,12,26,3000.000000,0.000000,0,26,0,1,15.954331
4,46272,317,1.0,157.000000,1.0,2016,12,26,157.000000,0.000000,0,26,0,1,15.954331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35124,40370,1380,0.0,1000.000000,1.0,2019,11,25,1000.000000,0.000000,0,25,0,1,12.638889
35125,40372,1380,0.0,2000.000000,1.0,2019,11,25,2000.000000,0.000000,0,25,0,1,6.722222
35126,40373,1380,0.0,3000.000000,1.0,2019,11,25,3000.000000,0.000000,0,25,0,1,4.166667
35127,46272,1380,1.0,284.290000,1.0,2019,11,25,199.000000,0.300011,0,25,0,1,4.451613


In [73]:
y

0        65.0
1        83.0
2        30.0
3         7.0
4        35.0
         ... 
35124    24.0
35125    11.0
35126     3.0
35127     0.0
35128     2.0
Name: Demand, Length: 33959, dtype: float64

In [74]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [75]:
from sklearn.metrics import make_scorer, mean_absolute_error
from sklearn.model_selection import cross_val_score

# подберем гиперпараметры на кросс-валидации

In [ ]:
# # подберем гиперпараметры на кросс-валидации
# from sklearn.model_selection import RandomizedSearchCV
# import numpy as np

# # Определим диапазоны гиперпараметров
# param_distributions = {
#     'max_depth': np.arange(3, 11, 1),  # Глубина деревьев от 3 до 10
#     'n_estimators': np.arange(100, 501, 50),  # Количество деревьев от 100 до 500 с шагом 50
#     'learning_rate': np.linspace(0.01, 0.2, 10),  # Скорость обучения от 0.01 до 0.2 (10 равномерных значений)
#     'min_samples_leaf': np.arange(5, 21, 2),  # Минимальное количество объектов в листе (от 5 до 20 с шагом 2)
#     'min_samples_split': np.arange(2, 16, 2),  # Минимальное количество объектов для разбиения (от 2 до 15 с шагом 2)
#     'subsample': np.linspace(0.7, 1.0, 5)  # Доля данных для обучения каждого дерева (от 0.7 до 1.0)
# }

# # RandomizedSearchCV для подбора гиперпараметров
# random_search = RandomizedSearchCV(
#     estimator=GradientBoostingRegressor(loss='absolute_error', random_state=1),
#     param_distributions=param_distributions,
#     n_iter=100,  # Количество итераций (случайных наборов параметров)
#     cv=5,  # Кросс-валидация
#     scoring='neg_mean_absolute_error',
#     verbose=1,
#     random_state=1,
#     n_jobs=-1  # Используем все ядра процессора
# )

# # Выполним поиск лучших гиперпараметров на тренировочных данных
# random_search.fit(X_train, y_train)

# # Выведем лучшие параметры
# print("Лучшие гиперпараметры:", random_search.best_params_)

# # Применяем лучшую модель
# best_model = random_search.best_estimator_

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Лучшие гиперпараметры: {'subsample': 0.925, 'n_estimators': 500, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_depth': 10, 'learning_rate': 0.1366666666666667}


- Лучшие гиперпараметры: {'subsample': 0.925, 'n_estimators': 500, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_depth': 10, 'learning_rate': 0.1366666666666667}

In [ ]:
# # Предсказания на тестовых данных
# y_pred_best = best_model.predict(X_test)

# # Оценка на тестовых данных
# mae_best = mean_absolute_error(y_test, y_pred_best)
# print(f"MAE на тестовых данных: {mae_best}")

MAE на тестовых данных: 5.115644501952501


- MAE на тестовых данных: 5.115644501952501

Попробуем еще немного подобрать гиперпараметры.

In [57]:
import pandas as pd

# Пример использования One-Hot Encoding для всех категориальных столбцов
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

# Дополнительно убедитесь, что тренировочный и тестовый наборы данных имеют одинаковые столбцы
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

In [76]:
X_train['weekday'].unique()

array([0], dtype=int32)

In [58]:
X_train

,SKU_id,Store_id,Promo,Regular_Price,AUTORIZATION_FLAG,ind_of_year,ind_of_month,ind_of_day,Actual_Price,Promo_percent,weekday,monthday,is_weekend,month_period,demand_expanding_mean
33898,40373,1347,0.0,3000.0,1.0,2019,2,25,3000.0,0.0,0,25,0,1,14.871020
33171,40372,1328,0.0,2000.0,1.0,2018,11,26,2000.0,0.0,0,26,0,1,17.478413
6878,46272,525,0.0,199.0,1.0,2018,7,9,199.0,0.0,0,9,0,0,10.509950
30970,40373,1281,0.0,3000.0,1.0,2019,3,25,3000.0,0.0,0,25,0,1,22.407109
20925,40372,1005,0.0,2000.0,1.0,2019,5,27,2000.0,0.0,0,27,0,1,17.219710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8053,40372,533,0.0,2000.0,0.0,2019,5,6,2000.0,0.0,0,6,0,0,5.572827
33621,40372,1347,0.0,2000.0,1.0,2018,4,9,2000.0,0.0,0,9,0,0,62.500000
5342,96212,504,1.0,99.0,1.0,2019,6,24,69.3,0.3,0,24,0,1,7.343293
12562,40370,562,0.0,1000.0,1.0,2019,2,4,1000.0,0.0,0,4,0,0,25.596924


In [77]:
#оптимальные параметры модели были подобраны
#обучим модель и проверим ее качество
regressor = GradientBoostingRegressor(
    max_depth=9,
    n_estimators=400,
    learning_rate=0.1,
    random_state=1,
    min_samples_leaf=5,
    min_samples_split=10,
    loss='absolute_error'
)
regressor.fit(X_train, y_train)

GradientBoostingRegressor(loss='absolute_error', max_depth=9,
                          min_samples_leaf=5, min_samples_split=10,
                          n_estimators=400, random_state=1)

In [78]:
y_pred = regressor.predict(X_test)

In [79]:
#предскажем искомые значения спроса для нашей тестовой выборки (с декабря 2019)
X_test = data_test.drop(['id', 'predicted'], axis=1)
X_test

,SKU_id,Store_id,Promo,Regular_Price,AUTORIZATION_FLAG,ind_of_year,ind_of_month,ind_of_day,Actual_Price,Promo_percent,weekday,monthday,is_weekend,month_period,demand_expanding_mean
888,40369,317,0.0,500.00,1.0,2019,12,2,500.0,0.000000,0,2,0,0,17.912489
889,40370,317,0.0,1000.00,1.0,2019,12,2,1000.0,0.000000,0,2,0,0,20.153431
890,40372,317,0.0,2000.00,1.0,2019,12,2,2000.0,0.000000,0,2,0,0,6.938798
891,40373,317,0.0,3000.00,1.0,2019,12,2,3000.0,0.000000,0,2,0,0,4.283099
892,46272,317,1.0,284.29,1.0,2019,12,2,199.0,0.300011,0,2,0,0,5.092526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35154,40370,1380,0.0,1000.00,1.0,2019,12,30,1000.0,0.000000,0,30,0,1,12.170732
35155,40372,1380,0.0,2000.00,1.0,2019,12,30,2000.0,0.000000,0,30,0,1,6.560976
35156,40373,1380,0.0,3000.00,1.0,2019,12,30,3000.0,0.000000,0,30,0,1,4.073171
35157,46272,1380,1.0,284.29,1.0,2019,12,30,199.0,0.300011,0,30,0,1,4.194444


In [80]:
#предскажем значения спроса для тестовой выборки
y_pred_res = regressor.predict(X_test)
y_pred_res

array([ 6.44875132,  8.20416448,  5.0330362 , ..., 14.83147719,
        4.62703393,  9.58080325])

In [81]:
#создадим датафрейм y_results, в который поместим результаты
y_results = data_test[['id', 'predicted']]
y_results['predicted'] = y_pred_res
y_results

/tmp/ipykernel_552303/2383451893.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_results['predicted'] = y_pred_res


,id,predicted
888,908,6.448751
889,909,8.204164
890,910,5.033036
891,911,4.120091
892,912,3.005649
...,...,...
35154,35537,64.375897
35155,35538,29.750611
35156,35539,14.831477
35157,35540,4.627034


In [82]:
# удалим отрицательыне прогнозы из выборки
y_results.loc[y_results['predicted'] < 0, ['predicted']]

,predicted
14660,-0.613473
24765,-0.880815


In [83]:
#заменим все отрицательные предсказанные значения спроса на ноль
y_results.loc[y_results['predicted'] < 0, ['predicted']] = 0
y_results.loc[y_results['predicted'] < 0, ['predicted']]

,predicted


In [84]:
#Записываем полученный датафрейм в csv файл:
y_results.to_csv('submission_res45.csv',sep=',', encoding='utf-8', index=False)